In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('Project1_Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#!kaggle competitions download -c da218o-class-competition-predict-missing-data



Project1_Data/coordinates.csv
Project1_Data/Node_501.csv
Project1_Data/Node_502.csv
Project1_Data/Node_505.csv
Project1_Data/Node_507.csv
Project1_Data/Node_508.csv
Project1_Data/Node_locations.xlsx
Project1_Data/sample_submission.csv
Project1_Data/submission_file.csv


In [94]:
pd1 = pd.read_csv('Project1_Data/Node_501.csv', index_col=0)
pd1['lat'] = -37.8148084
pd1['lng'] = 144.980999

pd2 = pd.read_csv('Project1_Data/Node_502.csv', index_col=0)
pd2['lat'] = -37.8146097
pd2['lng'] = 144.9790177

pd3 = pd.read_csv('Project1_Data/Node_505.csv', index_col=0)
pd3['lat'] = -37.813073
pd3['lng'] = 144.9804061

pd4 = pd.read_csv('Project1_Data/Node_507.csv', index_col=0)
pd4['lat'] = -37.8149218
pd4['lng'] = 144.9822582

pd5 = pd.read_csv('Project1_Data/Node_508.csv', index_col=0)
pd5['lat'] = -37.8134078
pd5['lng'] = 144.9794923



In [95]:
# center coordinate of all 5 sensors
lat0 = -37.8141641 #average of all 5 latitutes
lng0 = 144.9804347 #average of all 5 longitutude

In [96]:
spd = pd.concat([pd1, pd2,pd3,pd4,pd5], axis=0)
# spd = pd.concat([pd2], axis=0)
# spd.set_index('ID')

## Data preprocessing

In [97]:
spd['ts'] = spd.timestamp.apply(lambda x: pd.to_datetime(x.replace('/0014','/2014').replace('/0015','/2015'),format='%d/%m/%Y %H:%M'))
spd['ts_value'] = spd.ts.apply(lambda x: x.value/10**9)
spd['mins'] = spd.ts.apply(lambda x: (x.hour)*60 + x.minute-1)
min_maximum = spd.describe()['mins']['max']
spd['date'] = spd.ts.apply(lambda x: x.day-1 + (x.month-1)*30 + (x.year-2014)*30*12)
spd['sin_minute'] = spd.mins.apply(lambda x: math.sin(x*2*math.pi/min_maximum))
spd.shape

(6583, 10)

In [98]:
# spd.set_index('ID')
spd.head()
# spd.index

,timestamp,temperature,humidity,lat,lng,ts,ts_value,mins,date,sin_minute
ID,,,,,,,,,,
1,09/12/0014 00:03,25.16,36.47,-37.814808,144.980999,2014-12-09 00:03:00,1.418083e+09,2,338,0.008739
2,09/12/0014 00:13,24.83,35.52,-37.814808,144.980999,2014-12-09 00:13:00,1.418084e+09,12,338,0.052409
3,09/12/0014 00:23,24.83,35.68,-37.814808,144.980999,2014-12-09 00:23:00,1.418085e+09,22,338,0.095979
4,09/12/0014 00:33,24.83,35.84,-37.814808,144.980999,2014-12-09 00:33:00,1.418085e+09,32,338,0.139365
5,09/12/0014 00:44,24.83,35.99,-37.814808,144.980999,2014-12-09 00:44:00,1.418086e+09,43,338,0.186780


In [109]:
tnan = (spd['temperature'].notna())
hnan = (spd['humidity'].notna())
#tnan.head()
#hnan.head()
spd_temp = spd[tnan==False]
spd_humi = spd[hnan==False]

In [110]:
print(spd_temp.shape)
spd_temp.head()

(238, 10)


,timestamp,temperature,humidity,lat,lng,ts,ts_value,mins,date,sin_minute
ID,,,,,,,,,,
20,09/12/0014 03:18,NaN,38.04,-37.814808,144.980999,2014-12-09 03:18:00,1.418095e+09,197,338,0.758345
119,09/12/0014 20:14,NaN,37.10,-37.814808,144.980999,2014-12-09 20:14:00,1.418156e+09,1213,338,-0.832227
149,10/12/0014 09:15,NaN,41.67,-37.814808,144.980999,2014-12-10 09:15:00,1.418203e+09,554,339,0.660098
155,10/12/0014 10:17,NaN,41.35,-37.814808,144.980999,2014-12-10 10:17:00,1.418207e+09,616,339,0.435008
163,10/12/0014 11:39,NaN,41.67,-37.814808,144.980999,2014-12-10 11:39:00,1.418212e+09,698,339,0.091629


In [111]:
print(spd_humi.shape)
spd_humi.head()

(262, 10)


,timestamp,temperature,humidity,lat,lng,ts,ts_value,mins,date,sin_minute
ID,,,,,,,,,,
56,09/12/0014 09:27,23.87,NaN,-37.814808,144.980999,2014-12-09 09:27:00,1.418117e+09,566,338,0.619823
67,09/12/0014 11:20,23.54,NaN,-37.814808,144.980999,2014-12-09 11:20:00,1.418124e+09,679,338,0.173887
97,09/12/0014 16:28,22.90,NaN,-37.814808,144.980999,2014-12-09 16:28:00,1.418142e+09,987,338,-0.921139
166,10/12/0014 12:10,23.87,NaN,-37.814808,144.980999,2014-12-10 12:10:00,1.418213e+09,729,339,-0.043680
194,10/12/0014 16:57,23.22,NaN,-37.814808,144.980999,2014-12-10 16:57:00,1.418231e+09,1016,339,-0.962943


### Baseline estimate

In [147]:
def find_next(target, id_):
    a = spd.loc[id_+1]
    if(math.isnan(a[target])):
        return find_next(target,id_+1)
    return a

def find_prev(target, id_):
    a = spd.loc[id_-1]
    if(math.isnan(a[target])):
        return find_prev(target,id_-1)
    return a

In [149]:
result = pd.DataFrame()

for i, row in spd_temp.iterrows():
    b = find_next('temperature',i)
    a = find_prev('temperature',i)
    if(math.isnan(a['temperature'])):
        ts = b['temperature']
    elif(math.isnan(b['temperature'])):
        ts = a['temperature']
    else:
#         print("i == ",i,"\n a ==")
#         print(a)
#         print(b)
        ts = (a['temperature']*(b['ts_value']-row['ts_value']) + b['temperature']*(row['ts_value']-a['ts_value']))/(-a['ts_value']+b['ts_value'])
    df2 = {'ID': i, 'Predictions': ts}
    result = result.append(df2, ignore_index = True)
    
for i, row in spd_humi.iterrows():
    b = find_next('humidity',i)
    a = find_prev('humidity',i)
    if(math.isnan(a['humidity'])):
        ts = b['humidity']
    elif(math.isnan(b['humidity'])):
        ts = a['humidity']
    else:
        ts = (a['humidity']*(b['ts_value']-row['ts_value']) + b['humidity']*(row['ts_value']-a['ts_value']))/(-a['ts_value']+b['ts_value'])
    df2 = {'ID': i, 'Predictions': ts}
    result = result.append(df2, ignore_index = True)
    
result['ID'] = result['ID'].astype(int)
result.columns

C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resul

C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resul

C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resul

C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resul

C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)
C:\Users\Archit Agarwal\AppData\Local\Temp\ipykernel_12852\2604537583.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(df2, ignore_index = True)


Index(['ID', 'Predictions'], dtype='object')

In [150]:
result.sort_values("ID")
result.shape

(500, 2)

In [151]:
result.to_csv("average_baseline.csv",index=False)